# Step 3

## Use the projection to gradient space to predict task condition

Here we take the reconstructed time series, average within each condition and try to predict the condition in new subjects.

In [1]:
import numpy as np
import h5py as h5

from fgrad.predict import features_targets

/usr/local/lib/python2.7/site-packages/nibabel/cifti2/parse_cifti2.py:24: FutureWarning: We no longer carry a copy of the 'six' package in nibabel; Please import the 'six' package directly
  from ..externals.six import BytesIO


In [2]:
#f = h5.File('../data/reconstructed_WM.hdf5')
#d_LR = f['Working_memory/LR']
#d_RL = f['Working_memory/RL']

In [3]:
f = h5.File('/Users/marcel/projects/HCP/volumes_embedded_full.hdf5')
d_LR = f['Working_memory/Run1']
d_RL = f['Working_memory/Run2']

## Within-run averaging

We take the time points corresponding to each task condition and average them independently within each run.

In [4]:
labels = dict()
labels['WM_fix'] = 0
labels['WM_0back'] = 1
labels['WM_2back'] = 2

In [5]:
# Block onsets expressed as TRs
# We add 6 volumes (4.32 s) to each onset to take into account hemodynamic lag 
# and additional 4 volumes (2.88 s) to account for instruction
nback_LR_2b = np.round(np.array([7.977, 79.369, 150.553, 178.689])/0.72).astype(int)+10
nback_LR_0b = np.round(np.array([36.159, 107.464, 221.965, 250.18])/0.72).astype(int)+10
nback_RL_2b = np.round(np.array([7.977, 79.369, 178.769, 250.22])/0.72).astype(int)+10
nback_RL_0b = np.round(np.array([36.159, 107.464, 150.567, 222.031])/0.72).astype(int)+10
nback_fix = np.array([88, 187, 286])+6

# Each block lasts for 27.5 seconds
vols_2b_LR = np.concatenate([range(x,x+38) for x in nback_LR_2b])
vols_0b_LR = np.concatenate([range(x,x+38) for x in nback_LR_0b])
vols_2b_RL = np.concatenate([range(x,x+38) for x in nback_RL_2b])
vols_0b_RL = np.concatenate([range(x,x+38) for x in nback_RL_0b])
vols_fix = np.concatenate([range(x,x+22) for x in nback_fix])
vols_fix = np.concatenate([vols_fix, range(395, 405)])

# Targets
nback_targets_LR = np.zeros(405)
nback_targets_LR[vols_2b_LR] = 1
nback_targets_LR[vols_fix] = -1

nback_targets_RL = np.zeros(405)
nback_targets_RL[vols_2b_RL] = 1
nback_targets_RL[vols_fix] = -1

In [6]:
# Get random group assignments
subjects = f['Working_memory/Subjects'][...]
np.random.seed(123)
sind = np.arange(len(subjects))
G1 = sorted(np.random.choice(sind, 100, replace = False ))
sind = np.delete(sind,G1)
G2 = sorted(np.random.choice(sind, 100, replace = False ))
sind = np.delete(sind,G2)
G3 = sorted(np.random.choice(sind, 100, replace = False ))

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:8: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.


## Prepare the features

In [7]:
conds = ['WM_fix', 'WM_0back', 'WM_2back']
grads = [0,1,2]

# Group 1

f_WM_train1, t_WM_train1 = features_targets(data = d_LR, subjects = G1, 
                                      inds = nback_targets_LR, condnames = conds, gradients = grads, labels = labels)

f_WM_train2, t_WM_train2 = features_targets(data = d_RL, subjects = G1, 
                                      inds = nback_targets_RL, condnames = conds, gradients = grads, labels = labels)

# Group 2

f_WM_train3, t_WM_train3 = features_targets(data = d_LR, subjects = G2, 
                                      inds = nback_targets_LR, condnames = conds, gradients = grads, labels = labels)

f_WM_train4, t_WM_train4 = features_targets(data = d_RL, subjects = G2, 
                                      inds = nback_targets_RL, condnames = conds, gradients = grads, labels = labels)

# Group 3

f_WM_test1, t_WM_test1 = features_targets(data = d_LR, subjects = G3, 
                                      inds = nback_targets_LR, condnames = conds, gradients = grads, labels = labels)

f_WM_test2, t_WM_test2 = features_targets(data = d_RL, subjects = G3, 
                                      inds = nback_targets_RL, condnames = conds, gradients = grads, labels = labels)

# Classify the task conditions

#### Assemble the training and test sets

In [12]:
f_WM_train = np.vstack([f_WM_train1, f_WM_train2, f_WM_train3, f_WM_train4])
t_WM_train = np.concatenate([t_WM_train1, t_WM_train2, t_WM_train3, t_WM_train4])

f_WM_test = np.vstack([f_WM_test1, f_WM_test2])
t_WM_test = np.concatenate([t_WM_test1, t_WM_test2])

#### Cross-validate to get optimal parameters

In [37]:
from sklearn.model_selection import GroupShuffleSplit, cross_val_score, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

groups = np.concatenate([np.ones(600),np.ones(600)+1])

p_grid = {"linearsvc__C": np.linspace(0.01,1.01,100)}
svc = LinearSVC(C=0.25, penalty="l1", dual=False)
scl = StandardScaler()
svc_z = make_pipeline(scl, svc)

gss = GroupShuffleSplit(n_splits=100, train_size=0.5, random_state=0)
gscv = GridSearchCV(svc_z, param_grid=p_grid, cv=gss, n_jobs = -1)

gscv.fit(f_WM_train, t_WM_train, groups = groups)

GridSearchCV(cv=GroupShuffleSplit(n_splits=100, random_state=0, test_size=0.2, train_size=0.5),
       error_score='raise',
       estimator=Pipeline(steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('linearsvc', LinearSVC(C=0.25, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l1', random_state=None, tol=0.0001,
     verbose=0))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'linearsvc__C': array([ 0.01  ,  0.0201, ...,  0.9999,  1.01  ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

### Examine the performance on independent data

In [48]:
from sklearn.metrics import confusion_matrix, classification_report

print "Test accuracy:", gscv.best_estimator_.score(f_WM_test, t_WM_test)
print "Confusion matrix:\n\n", confusion_matrix(t_WM_test, gscv.best_estimator_.predict(f_WM_test))
print "\n\nClassification report:\n\n", classification_report(t_WM_test, gscv.best_estimator_.predict(f_WM_test))

Test accuracy: 0.943333333333
Confusion matrix:

[[197   3   0]
 [  2 181  17]
 [  0  12 188]]


Classification report:

             precision    recall  f1-score   support

          0       0.99      0.98      0.99       200
          1       0.92      0.91      0.91       200
          2       0.92      0.94      0.93       200

avg / total       0.94      0.94      0.94       600



In [49]:
f.close()